### get geography information (area and land area) from Census Tigerweb, pull in population and housing Decennial Census counts and calculate population and housing density in 2010 and 2020

In [1]:
import sys
import pandas as pd
import numpy as np

In [8]:
sys.path.append('..')
import geo_getter as get

## urban villages - blocks

## maricopa places

In [11]:
plc10 = get.make_maricopa_pl('2010')
plc20 = get.make_maricopa_pl('2020')
plcs = pd.merge(plc10,plc20,how='left',on='GEOID')
plcs.rename(columns={'BASENAME10':'name'},inplace=True)
plcs = plcs.drop(['GEOID','AREAWATER10','BASENAME20','AREAWATER20'],axis=1)

## maricopa

In [12]:
mar10 = get.make_maricopa('2010')
mar20 = get.make_maricopa('2020')

In [13]:
mar = pd.merge(mar10,mar20,how='left',on='GEOID')

In [14]:
mar.rename(columns={'BASENAME10':'name'},inplace=True)
mar = mar.drop(['GEOID','AREAWATER10','BASENAME20','AREAWATER20'],axis=1)

## arizona

In [15]:
az10 = get.make_az('2010')
az20 = get.make_az('2020')
az = pd.merge(az10,az20,how='left',on='GEOID')
az['name'] = 'AZ'

In [16]:
az = az.drop(['GEOID','AREAWATER10','BASENAME10','BASENAME20','AREAWATER20'],axis=1)

## us

In [17]:
us10 = get.make_us('2010')
us20 = get.make_us('2020')
us = pd.merge(us10,us20,how='left',on='FUNCSTAT')

In [18]:
us['name'] = 'US'
us = us.drop(['FUNCSTAT','AREAWATER10','AREAWATER20'],axis=1)

## make big geography table - no urban vil

In [19]:
geos = pd.concat([blk,mar,az,us,plcs,comps,uv])

## read in pop and housing from output folder

In [20]:
data = pd.read_csv('output/dec_pop_hou_race.csv')

In [21]:
data = data[['name','Hou_10E','Hou_20E','Pop_10E','Pop_20E']]

## calc housing and pop density in 2010 and 2020

In [22]:
geos = pd.merge(data,geos,how='left',on='name')

In [23]:
geos['HouDen_10'] = geos.Hou_10E / geos.AREALAND_ACRE10
geos['PopDen_10'] = geos.Pop_10E / geos.AREALAND_ACRE10
geos['HouDen_20'] = geos.Hou_20E / geos.AREALAND_ACRE20
geos['PopDen_20'] = geos.Pop_20E / geos.AREALAND_ACRE20

In [24]:
geos.to_csv('output/dec_pop_hou_den.csv',index=False)